<a href="https://colab.research.google.com/github/vbertalan/LOG6308-TP3/blob/main/TP3_Squelette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LOG6308
# TP3 : Systèmes de recommandation et réseaux de neurones

L'objectif du TP3 est de vous familiariser avec la librairie `Tensorflow` et `Tensorflow Recommenders`. Nous souhaitons aussi vous familiariser avec le concept de réseaux neuronaux.
C'est pourquoi nous vous proposons d'effectuer des recommandations de films sur la base de données que vous connaissez bien maintenant : [MovieLens 100k](https://grouplens.org/datasets/movielens/).<br>

Le TP sera noté **sur 100**.

## Critères de correction

- Démarche valide et bien expliquée
- Réponses correctes et commentées
- Présentation soignée 
- Choix de fonctionnalités adéquat
- Interprétation étayée des résultats

## Instructions Globales

Le travail doit être fait en **équipe de deux**.

Vous avez le droit d'utiliser **seulement** les **librairies importées** pour résoudre les **questions 1, 2 et 3**. Si vous utilisez d’autres librairies, vos réponses ne seront pas considérées.

Vous pouvez répondre aux sous-questions en commentaire ou dans des cellules textes en prenant bien soin d’identifier à quelle question vous répondez.
Ceux qui le souhaite peuvent développer en local et écrire votre code dans des fichiers Python en `.py`. Ceci dit, j'attends de vous un README.md m'expliquant comment exécuter votre code avec une liste de dépendances (Requirements).

Pour les questions 1-2-3, le Notebook est suffisant. Vous pouvez marquer vos commentaires et réponses qualitatives dans des cellules textes. 
Par contre, pour la question 4, il est recommandé de fournir un rapport séparé du code en format PDF. Mais, si vous ne souhaitez pas rédiger de rapport, vous pouvez rédiger votre état de l’art et votre démarche dans des cellules textes du Notebook sur Colab.


Pour la remise du travail sur Moodle, on s’attend à un Zip qui contient un notebook en `.ipynb` et/ou des fichiers Python en `.py`. **Si vous décidez** de **rédiger un rapport** pour la **question 4**, vous devez alors aussi **inclure** un fichier **PDF**. 


### Comment télécharger le notebook

- Cliquez sur le menu "Fichier" (*File* en anglais) dans le coin supérieur gauche.
- Une fenêtre popup apparaît, trouvez `Télécharger -> Télécharger le fichier .ipynb` et cliquez dessus.


#### Installation de tensorflow datasets, tensorflow recommenders, et importation des librairies requises pour le TP

In [ ]:
!pip install tfds-nightly
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets==4.3
!pip install -q scann

     |████████████████████████████████| 4.2 MB 6.6 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 462 kB 18.9 MB/s 
     |████████████████████████████████| 3.9 MB 7.6 MB/s 
     |████████████████████████████████| 10.7 MB 5.9 MB/s 


In [ ]:
import os
import pprint
import tempfile

from tqdm import tqdm
from typing import Dict, Text

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

## Utilisation du GPU
Les calculs seront plus rapides si vous utilisez le GPU. Ça sera particulièrement important pour la dernière partie. Pour s'assurer que le notebook utilise le GPU, vous pouvez modifier la configuration ainsi :
* (EN) `Edit > Notebook Settings`
* (FR) `Modifier > Paramètres du notebook`

Par contre, faites attention à ne pas utiliser le GPU si vous n'en avez pas besoin. Colab limite le temps d'utilisation des GPUs pour sa version gratuite.

In [ ]:
#Test CPU ou GPU
if(len(tf.config.list_physical_devices('GPU')) == 0):
    print("Vous utilisez actuellement le CPU")
else:
    print("Vous utilisez actuellement le GPU")

#### Téléchargement de MovieLens 100k

Vous pouvez accéder à la documentation en appuyant sur ce [lien](https://www.tensorflow.org/datasets/catalog/movielens#movielens100k-ratings).

In [ ]:
# Les votes + des données supplémentaires
ratings = tfds.load("movielens/100k-ratings", split="train", shuffle_files = False)
# Les genres, titres et identifiants des films.
films = tfds.load("movielens/100k-ratings", split="train", shuffle_files = False)

In [ ]:
#tfds
type(ratings)

Comme vous le voyez, `ratings` et `films` sont générés par `tfds` et sont des `tf.data.Dataset`. Pour avoir une idée de comment les utilisées, vous pouvez consulter ces liens : <br>
- [DataSet](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)
- [tfds](https://www.tensorflow.org/datasets/overview)

In [ ]:
# Exemple d'utilisation
list(ratings.map(lambda x: x["user_id"]).take(10))

## Question 1 (15 pts)
Dans cette question, nous allons définir et entrainer un modèle dit *Two Tower* afin de prédire les votes selon cette formule :

$$pred_{i,j}= b + E_{u_i}^TE_{f_j}.$$


Où $(E_{u_i}, E_{f_j}) \in \mathbb{R}^n \times \mathbb{R}^n$ sont respectivement les plongements (<i>embeddings</i>) de l'utilisateur $i$, $u_i$, et du film $j$, $f_j$. De plus, $b \in \mathbb{R}$ est la constante qui représente la moyenne. Enfin, $n \in \mathbb{N}$ est respectivement la dimension de l'espace latent des utilisateurs et des films (dans cette question, $n=32$).

<br>

***Pour répondre aux questions, vous devez remplacer les `?` par les valeurs adéquates.***

### 1.1. Extraire les attributs nécessaires pour entrainer le modèle (1 pt)

On vous demande d'extraire des données les `titres de films`, les `identifiants utilisateurs`, les `votes`, et les `horodatages` (<i>timestamps</i>). Les données doivent être sous format chaine de caractères (`str`). Voici la [doc](https://www.tensorflow.org/datasets/catalog/movielens#movielens100k-ratings). <br><br>
*À modifier si vous voulez inclure d'autres features pour les questions 3 et 4.*

In [ ]:
votes = ratings.map(lambda x: {"?": x["?"],"?": x["?"],"?": x["?"], "?": tf.strings.as_string(x["?"])})

### 1.2. Statistiques sur les données de `MovieLens 100k`

#### 1.2.a Affichez le nombre d'utilisateurs uniques (1 pt)

In [ ]:
#Titres des films
titres_films    = films.map(lambda x: x["?"]).batch(1000000)# On prend tous les films d'un coup
films_unique    = np.unique(np.concatenate(list(titres_films)))
nb_films_unique = films_unique.shape[0]
nb_films_unique

#### 1.2.b Affichez le nombre de films uniques (1 pt)

In [ ]:
#Identifiant des utilisateurs
id_utilisateurs = votes.map(lambda x: x["?"]).batch(1000000)# On prend tous les utilisateurs d'un coup
id_uniques      = np.unique(np.concatenate(list(id_utilisateurs)))
nb_id_uniques   = id_uniques.shape[0]
nb_id_uniques

#### 1.2.c Affichez le nombre de votes et les fréquences de paires (utilisateurs, films) uniques. Constatez-vous des anomalies ? Si oui, quelles sont-elles ? (3 pts)

### 1.3. Initialisation de la metrique RMSE de tfrs (1 pt)

Soit $y\in \mathbb{R}^N$ un vecteur de valeur de votes issue de la base de données d'entrainement, et $\hat{y} \in \mathbb{R}^N$ la prédiction de ces votes par notre modèle. Pour que notre modèle soit performant, nous aimerions bien que $\hat{y}$ ait quasiment les mêmes valeurs que $y$. On cherche donc à minimiser la perte suivante, qui est la **MSE** (*Mean Square Error*) :

<br>

$$\boxed{l(\hat{y}, y) = ||\hat{y}-y||_2^2 = \cfrac{1}{N}\underset{i=1}{\overset{N}{\sum }}|\hat{y} - y|^2}.$$

<br>

Initialisez la tâche avec la perte adéquate en utilisant `tfrs.tasks.Ranking`, voici la [doc](https://www.tensorflow.org/recommenders/api_docs/python/tfrs/tasks/Ranking). Il faut donc utiliser `tf.keras.losses.MeanSquaredError()` comme **perte**, et `tf.keras.metrics.RootMeanSquaredError()` comme **métrique**.

In [ ]:
task = ?

### 1.4. Définition du modèle Two Towers


#### 1.4.1. On définit la dimension de l'espace latent (taille des plongements) comme étant égale à 32. Pourquoi ne pas avoir choisi une dimension plus élevée ? (1 pt)<br>

<u>Réponse</u> :<br>
*Insérer votre réponse ici*

In [ ]:
embedding_dimension = 32

#### 1.4.2. Définir les couches de plongement pour les utilisateurs et les films (1 pt)

Pour initaliser les espaces de plongements, vous pouvez vous aider de la documentation de [tf.keras.layers.Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding).<br>

Pour comprendre `tf.keras.layers.experimental.preprocessing.StringLookup`, aidez-vous de la [doc](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup).

In [ ]:
def initialisation_embeddings(embedding_dimension, id_uniques, films_unique):
    user_model = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=?, mask_token=None),
                                    tf.keras.layers.Embedding(len(?) + 1,# Le +1 représente la constante $c$
                                                                ?)], name="User_Embedding")

    movie_model = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=?,mask_token=None),
                                    tf.keras.layers.Embedding(len(?) + 1, 
                                                                ?)], name="Movie_Embedding")  
    return user_model, movie_model

#### 1.4.3. Assemblez le modèle *Two Towers* (2 pts)

In [ ]:
class MovieLensModel(tfrs.models.Model):

  def __init__(self, embedding_dimension, id_uniques, films_unique, task):
    super().__init__()
    self.user_model, self.movie_model = initialisation_embeddings(embedding_dimension, id_uniques, films_unique)
    
    self.pred = tf.keras.layers.Dot(axes=1)
    
    self.task: tf.keras.layers.Layer = task

  def call(self, features):
    # TO DO
    pass

  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    
    return self.task(labels=features["user_rating"], predictions=self.call(features))

### 1.5. Entrainement du modèle
Dans cette partie, on entraine et test le modèle defini au dessus.

#### Définir les bases de données d'entrainement et de validation (proportion $80\%-20\%$).

In [ ]:
N          = len(votes)
batch_size = 8192 #2^13
prop       = 0.8
train_len  = tf.cast(N * prop, dtype=tf.int64)
valid_len   = tf.cast(N - train_len, dtype=tf.int64)


# shuffled = votes.shuffle(N, seed=73, reshuffle_each_iteration=False)

tf.random.set_seed(73)
train = votes.take(train_len).shuffle(train_len, seed=73, reshuffle_each_iteration=False).batch(batch_size)
valid = votes.skip(train_len).take(valid_len).shuffle(valid_len, seed=73, reshuffle_each_iteration=False).batch(batch_size)

#### 1.5.1. Initialisez le modèle, l'optimiseur et les modules de callback pour l'entrainement (2 pts)

In [ ]:
# On tire un exemple pour construire le graphe du modèle
feature = next(iter(train))
feature

In [ ]:
# On construit et affiche le modèle
Model = MovieLensModel(?, ?, ?, ?)
Model(feature)
Model.summary() # comment expliquez-vous le nombre de paramètres des couches embeddings ? (32*x)

On utilise comme optimiseur `Adam` (voir la [doc](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)) qui prend $0.01$ comme valeur pour son `learning_rate`. On vous demande aussi d'utiliser la stratégie *early stopping* pour entrainer votre modèle (voir les explications [ici](https://www.educative.io/edpresso/what-is-early-stopping)). Cette stratégie est implémentée par `Keras` comme un module *callback*, voir la [doc](https://keras.io/api/callbacks/). La **patience** doit être égale à $3$.

In [ ]:
# Création du dossier contenant les modèles entrainés
!mkdir Models/

# Compiler le modèle en ajoutant l'optimiseur Adam
Model.compile(optimizer=?)

my_callbacks = [
    ?,
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

#### 1.5.2. Entrainez le modèle sur **15 epochs** et afficher les résultats ainsi que la meilleure **RMSE** sur l'ensemble de validation. Y a-t-il surapprentissage ? (2 pts)

In [ ]:
#Entrainement du modèle sur 15 epochs
history_TwoTowers = Model.fit(?, epochs=?, validation_data=?, callbacks=?)

In [ ]:
def plot_history(history, model_name="Two Towers"):
    # summarize history for loss
    plt.plot(history.history['?'])
    plt.plot(history.history['?'])
    plt.title(model_name + ' Model RMSE')
    plt.ylabel('RMSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()
    print("\n\nBest RMSE on validation : {0:.4f}".format(min(history.history['?'])))

plot_history(history_TwoTowers, model_name="Two Towers")

L'overfitting n'a pas lieu de manière flagrande pendant les 15 premières epochs

## Question 2 (15 pts)

Modifier le modèle Two Tower pour prendre en compte le biais utilisateur et item (film). La nouvelle formule de prédiction est donc :

$$pred_{i,j}= \sigma(b + biais_{u_i} + biais_{f_j} +E_{u_i}^TE_{f_j}) \times (M_{vote} - m_{vote}) + m_{vote}$$

<br>

Où $biais_{u_i} \in \mathbb{R}$ est le biais associé à l'utilisateur $u_i$ et $biais_{f_j} \in \mathbb{R}$ le biais associé au film $f_j$. <br>

$\sigma: x \mapsto \cfrac{1}{1+e^{-x}}$ est la fonction sigmoid, elle est déjà implémentée par TensorFlow : `tf.math.sigmoid`.<br>

Et, $M_{vote}, m_{vote}$ sont respectivement le maximum et le minimum des votes utilisateurs. Dans notre cas, $M_{vote}=5$ et $m_{vote}=1$.

### Description du modèle Two Tower avec Biais

Le modèle Keras correspondant est légèrement plus complexe. En plus des plongements d'utilisateurs et de films avec lesquelles nous avons déjà travaillé, le modèle ci-dessous approxime le biais utilisateur ($biais_{u_i}$) et le biais film ($biais_{f_j}$) en plongeant l'utilisateur et le film dans un espace unidimensionnel. Nous ajoutons ensuite les deux biais au produit scalaire représentant l'interaction utilisateur-film. La fonction d'activation sigmoïde normalise et ramène la prédiction à l'intervalle $[0,1]$, qui est ensuite ramenée à l'intervalle de vote original $[m_{vote}, M_{vote}]$. D'ailleurs, le dropout doit être appliqué aux sorties des couches `user_model` et `movie_model`.


### Définissez, initialisez, entrainez, affichez et interprétez les résultats du modèle Two Tower modifié. Y a-t-il surapprentissage ?

Dans cette question, il vous ai conseillé d'utiliser Adam avec un `learning_rate`$=0.005$.

In [ ]:
#MovieLensModelWithBias Herite des attributs et des méthodes de MovieLensModel
class MovieLensModelWithBias(MovieLensModel):

  def __init__(self, embedding_dimension, id_uniques, films_unique, task, min_vote=1, max_vote=5):
    super().__init__(embedding_dimension, id_uniques, films_unique, task)

    self.min_vote, self.max_vote = min_vote, max_vote

    # Cette couche plonge dans un espace de dimension 1. Sa sortie est une constante qui représente le biais utilisateur.
    self.user_bias = user_model = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=id_uniques, mask_token=None),
                                    tf.keras.layers.Embedding(len(id_uniques) + 1, 1)], name="User_Bias")

    self.movie_bias = user_model = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=films_unique, mask_token=None),
                                    tf.keras.layers.Embedding(len(films_unique) + 1, 1)], name="Movie_Bias")

    self.user_dropout  = tf.keras.layers.Dropout(rate = 0.3, name="User_Dropout")
    self.movie_dropout = tf.keras.layers.Dropout(rate = 0.6, name="Movie_Dropout")


  def call(self, features):
    # TO DO 
    pass

In [ ]:
#Initialisez le modèle et afficher ses couches (summary)
Model_2 = ?
Model_2(feature)
Model_2.summary()

In [ ]:
# Compilez le modèle en ajoutant l'optimiseur Adam


In [ ]:
#Entrainez le modèle


In [ ]:
#Affichez les résultats


#### Observations

<u>Réponse</u> :<br>

*Répondez ici*

## Question 3 (20 pts)
Dans cette question, nous cherchons à améliorer le modèle Two Towers avec les biais de la question 2. <br> 

Voici quelques idées d'améliorations : 

### Question 3.1 (10 pts)

Améliorez les performances en changeant les hyperparamètres du modèle (<i>dropout, embedding_dim, learning rate, etc...</i>). Quelle est l'impact de ces hyperparamètres sur le surapprentissage (<i>overfitting</i>) ? **(10 pts)**

### Question 3.2 (10 pts)

Commencez l'entrainement du modèle avec des plongements pré-entrainés (pretrained embeddings) obtenus aux questions précédentes. **(10 pts)**

### Bonus (10 pts)

Prenez en compte les `timestamps`, ou développez d'autres idées que vous détaillerez.

## Question 4 (50 points)

Maintenant que vous vous êtes familiarisés avec les librairies de `Tensorflow`, attaquons-nous à l'état de l'art. En utilisant des mots-clés comme `Deep Learning`, `Recommender Systems`, et `MovieLens`, faites une brève revue de l'état de l'art. Il est impératif que vous <b>citiez vos [sources](https://ulyngs.github.io/oxforddown/cites-and-refs.html)</b>.

Ensuite, inspirez-vous de vos recherches pour proposer une approche plus performante que celle vue au-dessus. Pour cette question, il est recommandé de fournir un rapport séparé pour votre état de l'art et l'explication de votre démarche en format PDF. Mais, si vous ne souhaitez pas rédiger de rapport, vous pouvez rédiger dans les cellules textes ci-dessous.


<br>

Cette question vous laisse beaucoup de liberté dans vos réponses, vous pouvez utiliser n'importe quelle bibliothèque Python contrairement aux autres questions. Néanmoins, vous êtes quand même **soumis à des contraintes** :
- Vous devez utiliser uniquement les données `MovieLens 100k`
- **Si** vous **n**'avez **pas** besoin de features supplémentaires de `Movielens 100k` que celle extraite dans la **question 1.1**, utilisez l'ensemble d'entrainement (`train`) et de validation (`valid`) créée à la question 1.5
- Votre modèle doit se baser sur des réseaux de neurones
- Votre modèle doit être en tensorflow
- Vous **ne** pouvez **pas** **entrainer** vos modèles sur les données de **validation** 
- Citez obligatoirement vos sources !


<br>

Une approche possible qu'on vous propose est de **réimplémenter** la méthode décrite dans le papier [Scalable deep learning-based recommendation systems](https://www.sciencedirect.com/science/article/pii/S2405959518302029) de H. Lee et al.<br>

Pour cela il faut :
1. Créer la matrice utilisateur-item
2. Implémenter leur preprocessing sur la matrice utilisateur-item
3. Implémenter le modèle décrit pour `MovieLens 100k`
4. Entrainer le modèle
5. Comparer les résultats obtenus en calculant la RMSE sur l'ensemble de validation par rapport à ceux obtenus par les méthodes précédentes

<br><br>


<big><b><center>Les 3 groupes ayant les meilleurs RMSE sur l'ensemble de validation auront 10 points de bonus.</center></b></big>

<br>
<br>
<br>


**Qualités attendues du travail**, vous serez noté selon :
- L'originalité de votre démarche
- La cohérence de votre démarche avec l'état de l'art rédigé
- Les résultats empiriques (**RMSE**) sur l'ensemble de validation, notamment est-ce qu'elle performe mieux que les méthodes précédentes de manière consistante ?

#### État de l'art (15 points)



<u>Réponse :</u><br>
<i>Vous pouvez rédiger ici.</i>

#### Code et démarche (35 points)

##### Démarche et raisonnement :

<ul>
  <li>Citez vos sources qui vous aider à produire votre solution. En particulier, mentionnez la source du code que vous avez pris et modifié s'il y a lieu</li>
  <li>Expliquez votre démarche et votre raisonnement.</li>
</ul>


<u>Réponse :</u><br>
<i>Vous pouvez rédiger ici.</i>

##### Code

Faites en sorte que le code soit lisible et facilement interprétable.